# Анализ линейной зависимости между фичами  (часть 2)
мультиколлиниарность

Входные данные data/interm/target_features.pkl из feature_engineering.ipynb уже рассчитанные фичи

In [20]:
import pandas as pd

In [21]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb
%run ../preprocessing/preprocessing_target.ipynb
%run ../ML_functions.ipynb
%run ../auxiliary/plots.ipynb

In [22]:
name = pre + "data/interm/target_features.pkl"
target_features = pd.read_pickle(name, compression={'method':'zip'})
target_features['Метка дефолта'] = target_features['Метка фрод дефолта'].astype('int').astype('str')
print("Размер считанных данных target_class with mdm features- {}".format(target_features.shape))
# target_features.head(1)

Размер считанных данных target_class with mdm features- (85798, 40)


In [23]:
# сюда таблицы будем кидать, для отчетов
name = pre + "data/interm/auxilary/multicollinear.xlsx"
writer = ExcelWriter(name)

In [24]:
total_features = allFeaturesProperties.keys()
print(total_features)

dict_keys(['no_contracts_total', 'no_contracts_active', 'no_transport_total', 'no_transp_active', 'no_contracts_closed', 'reason_завершение', 'reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_расторгнут', 'reason_страховой случай', 'reason_страховой случай (гибель/тотал)', 'reason_гибель/тотал', 'bad_reasons_closed', 'good_reasons_closed', 'not_determined_reasons_closed', 'mean_bad_reasons', 'mean_good_reasons', 'mean_not_determined_reasons', 'no_creditors', 'duration_max', 'duration_min', 'duration_mean', 'contract_date_delta_first', 'contract_date_delta_last', 'end_date_last', 'end_date_first'])


In [25]:
#  timedelta с делаем в днях и числовыми значениями
target_features = days_substitute(target_features)

Поскольку корреляция  и мультиколлиниарность  очень чувствительна к выбросам, то заменим outliers медианным значениями. 

В случае, когда распределение более 95% около 0, то такое распределение будет считаться не содержит outliers. В корреляции будут участвовать все значения

In [26]:
keep_columns = ['Приложение', 'Метка дефолта', *total_features]

features_wo_outliers = target_features[keep_columns].copy()

for feature in keep_columns:
   if feature in ['Приложение', 'Метка дефолта']:
      continue

   percentile_max = 1 if feature in ['reason_замена/перенайм/переуступка', 'reason_определена цессия', 
                                        'reason_расторгнут', 'reason_страховой случай', 'reason_страховой случай (гибель/тотал)', 
                                        'reason_гибель/тотал', 'bad_reasons_closed'] else 0.95
                                        
   features_wo_outliers[feature] = replace_outliers_with_median_quantile(features_wo_outliers[feature], upper = percentile_max)

Interpreting VIF

The output gives us the VIF for each variable. A VIF close to 1 indicates that the variable is not correlated with the other variables, and hence its variance is not inflated at all. A VIF greater than 1 suggests the presence of multicollinearity.

As a rule of thumb, a VIF above 5 indicates a high multicollinearity between this variable and the others, and above 10 is very high multicollinearity.

A value between 1 and 5 indicates moderate correlation between a given explanatory variable and other explanatory variables in the model, but this is often not severe enough to require attention.

A value greater than 5 indicates potentially severe correlation between a given explanatory variable and other explanatory variables in the model. In this case, the coefficient estimates and p-values in the regression output are likely unreliable.


| VIF value | Diagnosis                                        |
| --------- | ------------------------------------------------ |
| 1         | Complete absence of multicollinearity            |
| 1-2       | Absence of strong multicollinearity              |
| > 2       | Presence of moderate to strong multicollinearity |

Note: There is no universal agreement of VIF values for multicollinearity detection. The VIF > 5 or VIF > 10 indicates strong multicollinearity, but VIF < 5 also indicates multicollinearity. It is advisable to have VIF < 2.


### Шаг 1. 
смотрим, увидит ли VIF мультиколлиниарность в следующих случаях:  
- 'bad_reasons_closed' = ['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)'].sum(axis=1)  
- ['good_reasons_closed'] = ['reason_завершение']  
- ['not_determined_reasons_closed'] = ['reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай'].sum(axis=1)   
- ['no_contracts_closed'] = ['not_determined_reasons_closed', 'bad_reasons_closed', 'good_reasons_closed'].sum(axis=1)  
- ['no_contracts_total'] = ['no_contracts_active', 'no_contracts_closed'].sum(axis=1) 

In [27]:
# Проверка, что фичи мультиколлинеарны
bad_collinear = np.all(target_features[['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)']].sum(axis=1) \
                        ==target_features['bad_reasons_closed'])

good_collinear = np.all(target_features['reason_завершение']==target_features['good_reasons_closed'])

not_determined_collinear = np.all(target_features[['reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай']].sum(axis=1) \
                        ==target_features['not_determined_reasons_closed'])

closed_collinear = np.all(target_features[['not_determined_reasons_closed', 'bad_reasons_closed', 'good_reasons_closed']].sum(axis=1) \
                        ==target_features['no_contracts_closed'])

lst = ['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)', 
      'reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай', 
      'reason_завершение']

contracts_total_collinear = np.all(target_features[['no_contracts_active', 'no_contracts_closed']].sum(axis=1) \
                        ==target_features['no_contracts_total'])


print('bad_collinear ', bad_collinear)      
print('good_collinear ', good_collinear)      
print('not_determined_collinear ', not_determined_collinear)           
print('closed_collinear ', closed_collinear)     
print('contracts_total_collinear ', contracts_total_collinear)     
      

bad_collinear  True
good_collinear  True
not_determined_collinear  True
closed_collinear  True
contracts_total_collinear  True


In [28]:
check_features = total_features
vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning:

divide by zero encountered in double_scalars



,VIF,variable
0,inf,no_contracts_total
9,inf,reason_отмена
16,inf,not_determined_reasons_closed
15,inf,good_reasons_closed
1,inf,no_contracts_active
13,inf,reason_гибель/тотал
12,inf,reason_страховой случай (гибель/тотал)
11,inf,reason_страховой случай
10,inf,reason_расторгнут
14,inf,bad_reasons_closed


Абсолютно верно методом VIF выявлены линейные зависимости у 15 фичей:
- 'reason_гибель/тотал' +, 'reason_расторгнут' +, 'reason_страховой случай (гибель/тотал)' +
- 'bad_reasons_closed' +
- 'reason_завершение'+ , 'good_reasons_closed'+
- 'reason_замена/перенайм/переуступка' +, 'reason_не указана' + , 'reason_определена цессия' + , 'reason_отмена' + , 'reason_страховой случай' + 
- 'not_determined_reasons_closed'+
- 'no_contracts_closed'+ 
- 'no_contracts_active'+
- no_contracts_total +

## Шаг 2. 

Анализ мультиколлиниарности на полных данных без фичей, исключенные ранее из-за нестабильности или не репрезентативности или коррелированности

In [29]:
#  список фичей, который был исключен из-за нестабильности и не репрезентативности
unstable_list = ['contract_date_delta_first', 'contract_date_delta_last','end_date_last']
#  список фичей, который был исключен из-за коррелированности
correlated_list = ['no_contracts_active', 'no_transp_active', 'reason_завершение', 'reason_не указана', 'reason_расторгнут']

In [30]:
check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list))
print(check_features)

vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'duration_max', 'duration_mean', 'reason_отмена', 'reason_страховой случай', 'not_determined_reasons_closed', 'reason_замена/перенайм/переуступка', 'no_transport_total', 'reason_гибель/тотал', 'no_contracts_closed', 'no_contracts_total', 'mean_not_determined_reasons', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'bad_reasons_closed', 'good_reasons_closed', 'end_date_first']


c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning:

divide by zero encountered in double_scalars



,VIF,variable
18,inf,good_reasons_closed
11,inf,no_contracts_closed
7,inf,not_determined_reasons_closed
17,inf,bad_reasons_closed
4,32.231498,duration_mean
3,20.020589,duration_max
12,14.865433,no_contracts_total
9,12.244726,no_transport_total
1,7.655410,duration_min
19,3.809311,end_date_first


После удаления нестабильных и коррелированных фичей, необходимо из групп линейнозависимых фичей удалить по одной фиче в группе ['no_contracts_closed', 'not_determined_reasons_closed', 'bad_reasons_closed', 'good_reasons_closed']
- not_determined_reasons_closed - исключается

In [31]:
#  список фичей, который был исключен из-за линейной зависимости
linear_depended_list = ['not_determined_reasons_closed']

In [32]:
check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list) - set(linear_depended_list))
print(check_features)

vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'duration_max', 'duration_mean', 'reason_отмена', 'reason_страховой случай', 'reason_замена/перенайм/переуступка', 'no_transport_total', 'reason_гибель/тотал', 'no_contracts_closed', 'no_contracts_total', 'mean_not_determined_reasons', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'bad_reasons_closed', 'good_reasons_closed', 'end_date_first']


,VIF,variable
4,32.231498,duration_mean
3,20.020589,duration_max
11,14.865433,no_contracts_total
8,12.244726,no_transport_total
1,7.655410,duration_min
10,6.404287,no_contracts_closed
18,3.809311,end_date_first
17,3.458999,good_reasons_closed
12,3.439439,mean_not_determined_reasons
2,2.667484,mean_good_reasons


## Шаг 3. 

Исключение фичей с высоким VIF на полных данных без ранее исключенных фичей

In [33]:
excluded = ['no_transport_total', 'duration_max', 'duration_mean', 'good_reasons_closed', 'mean_not_determined_reasons']

check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list) - set(linear_depended_list) - \
                        set(excluded))
                        
print(check_features)

vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'reason_гибель/тотал', 'no_contracts_closed', 'no_contracts_total', 'reason_отмена', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'reason_страховой случай', 'end_date_first', 'bad_reasons_closed', 'reason_замена/перенайм/переуступка']


,VIF,variable
4,4.352068,no_contracts_closed
5,3.932320,no_contracts_total
9,2.345272,no_creditors
11,2.060579,end_date_first
12,1.666157,bad_reasons_closed
6,1.614977,reason_отмена
2,1.449442,mean_good_reasons
1,1.361830,duration_min
8,1.272435,mean_bad_reasons
10,1.030002,reason_страховой случай


высокий VIF между коррелированными фичами no_contracts_closed и no_contracts_total, и с фичей  Разница до даты самого первого завершения договора  


In [34]:
check_features = ['no_contracts_closed', 'no_contracts_total', 'end_date_first']
vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

,VIF,variable
0,3.415484,no_contracts_closed
1,2.949655,no_contracts_total
2,1.385229,end_date_first


Дополнительное исключение 'end_date_first' не сильно улучшило VIF 'no_contracts_closed'

In [35]:
excluded = ['no_transport_total', 'duration_max', 'duration_mean', 'good_reasons_closed', 'mean_not_determined_reasons', 'end_date_first']

check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list) - set(linear_depended_list) - \
                        set(excluded))
                        
print(check_features)

vif_data = calc_vif(target_features[check_features])
vif_data.sort_values(by='VIF', ascending=False)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'reason_гибель/тотал', 'no_contracts_closed', 'no_contracts_total', 'reason_отмена', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'reason_страховой случай', 'bad_reasons_closed', 'reason_замена/перенайм/переуступка']


,VIF,variable
4,3.908222,no_contracts_closed
5,3.649085,no_contracts_total
9,1.950821,no_creditors
11,1.658786,bad_reasons_closed
6,1.564925,reason_отмена
1,1.355669,duration_min
2,1.352264,mean_good_reasons
8,1.257762,mean_bad_reasons
10,1.029365,reason_страховой случай
7,1.025105,reason_определена цессия


## Шаг 4. 

Анализ мультиколлиниарности на массиве данных без outliers 

In [36]:
check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list) - set(linear_depended_list))
                        
print(check_features)

vif_data = calc_vif(features_wo_outliers[check_features])
vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'duration_max', 'duration_mean', 'reason_отмена', 'reason_страховой случай', 'reason_замена/перенайм/переуступка', 'no_transport_total', 'reason_гибель/тотал', 'no_contracts_closed', 'no_contracts_total', 'mean_not_determined_reasons', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'bad_reasons_closed', 'good_reasons_closed', 'end_date_first']


,VIF,variable
0,3.686932,duration_mean
1,3.489016,no_contracts_total
2,3.253453,mean_not_determined_reasons
3,3.238838,no_transport_total
4,2.766393,duration_max
5,2.714864,mean_good_reasons
6,2.388798,duration_min
7,2.070903,no_contracts_closed
8,2.066556,end_date_first
9,1.964250,good_reasons_closed


In [37]:
excluded =['duration_mean', 'no_transport_total', 'mean_not_determined_reasons', 'end_date_first']
check_features = list(set(total_features) -  set(unstable_list) - set(correlated_list) - set(linear_depended_list) - \
                        set(excluded))
                        
print(check_features)

vif_data = calc_vif(features_wo_outliers[check_features])
vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)

['reason_страховой случай (гибель/тотал)', 'duration_min', 'mean_good_reasons', 'reason_гибель/тотал', 'duration_max', 'no_contracts_closed', 'no_contracts_total', 'reason_отмена', 'reason_определена цессия', 'mean_bad_reasons', 'no_creditors', 'reason_страховой случай', 'bad_reasons_closed', 'good_reasons_closed', 'reason_замена/перенайм/переуступка']


,VIF,variable
0,1.968217,duration_max
1,1.941457,no_contracts_closed
2,1.934977,good_reasons_closed
3,1.886444,mean_good_reasons
4,1.725325,no_creditors
5,1.637340,no_contracts_total
6,1.571579,duration_min
7,1.195916,reason_отмена
8,1.078077,mean_bad_reasons
9,1.041544,bad_reasons_closed


## Заключение из анализа мультиколлиниарности

В рассчитанных фичах есть пять линейнозависимых групп:

1. 'bad_reasons_closed' = ['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)'].sum(axis=1)  
2. ['good_reasons_closed'] = ['reason_завершение']  
3.  ['not_determined_reasons_closed'] = ['reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай'].sum(axis=1)   
4.  ['no_contracts_closed'] = ['not_determined_reasons_closed', 'bad_reasons_closed', 'good_reasons_closed'].sum(axis=1)  
5.  ['no_contracts_total'] = ['no_contracts_active', 'no_contracts_closed'].sum(axis=1) 
   
Если в анализе на мультиколлинеарность уже исключены нестабильные, нерепрезентативные и коррелированные фичи, а именно

- список фичей, который был исключен из-за нестабильности и не репрезентативности  
unstable_list = ['contract_date_delta_first', 'contract_date_delta_last','end_date_last']

- список фичей, который был исключен из-за коррелированности  
correlated_list = ['no_contracts_active', 'no_transp_active', 'reason_завершение', 'reason_не указана', 'reason_расторгнут']

, то необходимо исключить 'not_determined_reasons_closed', так как она входит в группу линейно зависимых  
'no_contracts_closed' = 'not_determined_reasons_closed' + 'bad_reasons_closed' + 'good_reasons_closed'

Дополнительные фичи ['duration_mean', 'no_transport_total', 'mean_not_determined_reasons', 'end_date_first'] были исключены из-за высокой степени мультиколлинеарности.

список оставшихся фичей:

|  | VIF |       variable                                 |
| --- | -------- | -------------------------------------- |
| 0   | 1.968960 | duration_max                           |
| 1   | 1.930499 | no_contracts_closed                    |
| 2   | 1.926102 | good_reasons_closed                    |
| 3   | 1.878113 | mean_good_reasons                      |
| 4   | 1.723763 | no_creditors                           |
| 5   | 1.635554 | no_contracts_total                     |
| 6   | 1.571514 | duration_min                           |
| 7   | 1.195993 | reason_отмена                          |
| 8   | 1.078484 | mean_bad_reasons                       |
| 9   | 1.041635 | bad_reasons_closed                     |
| 10  | 1.035443 | reason_гибель/тотал                    |
| 11  | 1.017040 | reason_страховой случай                |
| 12  | 1.007097 | reason_определена цессия               |
| 13  | 1.004203 | reason_страховой случай (гибель/тотал) |
| 14  | 1.000546 | reason_замена/перенайм/переуступка     |